In [ ]:
%matplotlib inline
import sys
sys.path.append("../JSON/")

import warnings
warnings.filterwarnings('ignore')

# LINQX imports
from models.parameter.vectorized import VParameterModel
from models.command.vectorized import VInteractiveResultDriverCommand
from models.workflow.vectorized import VBaseDriverWorkflow
from models.optimizer.vectorized import VBaseObjectiveFunction

# EvoTorch imports
from evotorch import Problem
from evotorch.algorithms import SteadyStateGA
from evotorch.operators import GaussianMutation

# Plotting imports
import matplotlib.pyplot as plt
import torch

In [ ]:
solvent_gradient_model = VParameterModel(
    name="SolventGradient",
    data_type="float",
    vector_type="tensor",
    upper_limit=8,
    lower_limit=4,
    default=[5,5,5,5,5],
    desc="Rate of solvent gradient per min"
)

flow_rate_model = VParameterModel(
    name="FlowRate",
    data_type="float",
    vector_type="tensor",
    upper_limit=500,
    lower_limit=100,
    default=[300,300,300,300,300],
)

injection_volume_model = VParameterModel(
    name="InjectionVolume",
    data_type="float",
    vector_type="tensor",
    upper_limit=1,
    lower_limit=0,
    default=[0,0,0,0,0]
)

SolventGradient = solvent_gradient_model.to_param()
FlowRate = flow_rate_model.to_param()
InjectionVolume = injection_volume_model.to_param()

test_sg = SolventGradient()
test_fr = FlowRate()
test_iv = InjectionVolume()

In [ ]:
print(test_sg.value)
print(test_fr.value)
print(test_iv.value)

In [ ]:
run_interactive_lc = VInteractiveResultDriverCommand(
    name="Interactive_LC_Run",
    parameters={
        "solvent_gradient": SolventGradient(),
        "flow_rate": FlowRate(),
        "injection_volume": InjectionVolume()
    },
    uuid="NEZ",
    result_vars=["resolution","other"],
)

In [ ]:
run_interactive_lc(
    solvent_gradient=[4,5,4,5,4],
    flow_rate=[200, 250, 300, 350, 400],
)

In [ ]:
print(run_interactive_lc._vector_length)

In [ ]:
run_interactive_lc(
    solvent_gradient=[4,4,4,4],
    flow_rate=[250,250,250,250],
    injection_volume=[0,0,0,0]
)

In [ ]:
print(run_interactive_lc._vector_length)

In [ ]:
lc_workflow = VBaseDriverWorkflow(
    name="interactive_vector_workflow",
    commands=[run_interactive_lc]
)

In [ ]:
lc_workflow.exec(list_kwargs=[
    {
        "solvent_gradient": [5,5,5,7,8],
        "flow_rate": [200,250,300,350,400],
        "injection_volume": [0,0,0,0,0]
    },
])

In [ ]:
obj_fn = VBaseObjectiveFunction(
    name="lc_obj_fn",
    workflow=lc_workflow,
    order_kwargs=[
        {
            "solvent_gradient": (0,1), 
            "flow_rate": (1,2),
            "injection_volume": (2,3)
        }
    ],
    list_save_vars=[{"resolution":"resolution", "other":"other"}],
    fitness_criteria=["resolution","other"]
)

In [ ]:
x = torch.Tensor(
        [
            [5,220,0],
            [6,250,0.1],
            [5,320,0.2],
            [4,340,0.3],
            [7,400,0.4]
        ]
    )

VBaseObjectiveFunction._assign_kwarg_list(
    order_kwargs=[
        {
            "solvent_gradient": (0,1), 
            "flow_rate": (1,2),
            "injection_volume": (2,3)
        }
    ],
    x=x
)

In [ ]:
obj_fn(x=x)

In [ ]:
bounds_pair = (
    (SolventGradient().lower_limit, FlowRate().lower_limit, InjectionVolume().lower_limit),
    (SolventGradient().upper_limit, FlowRate().upper_limit, InjectionVolume().upper_limit)
)

print(bounds_pair)

problem = Problem(
    objective_sense=["max","max"],
    objective_func=obj_fn,
    bounds=bounds_pair,
    solution_length=3,
    vectorized=True
)

In [ ]:
ga = SteadyStateGA(problem, popsize=2)
ga.use(
    GaussianMutation(problem, stdev=0.5)
)
ga.run(10)